In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
tf.random.set_seed(333)

xy_data = np.loadtxt("data-04-zoo.csv", delimiter=",", dtype=np.float32)
x_data = xy_data[:, :-1]  # 마지막 열을 제외한 모든 열
y_data = xy_data[:, -1]  # 마지막 열

print(x_data.shape,y_data.shape)

2.1.0
(101, 16) (101,)


In [2]:
nb_classes = 7  # 0~ 6 까지 분류
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)
W = tf.Variable(tf.random.normal((16, nb_classes)), name="Weight")
b = tf.Variable(tf.random.normal((nb_classes,)), name="Bias")
variables = [W, b]

print(Y_one_hot.shape)

(101, 7)


In [3]:
def logit(X):
    return tf.matmul(X, W) + b

# hypothesis - softmax
def hypothesis(X):
    return tf.nn.softmax(logit(X))


# cost -> using logit
def cost_fn(X, Y):
    cost = tf.keras.losses.categorical_crossentropy(
        y_true=Y, y_pred=logit(X), from_logits=True
    )
    return tf.reduce_mean(cost)


# gradient
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grad = tape.gradient(loss, variables)
        return grad


def predict(X, Y):
    # hypothesis(x).shape = (101, 7)
    pred = tf.argmax(hypothesis(X), 1)
    correct = tf.equal(pred, tf.argmax(Y, 1))
    # correct dtype = bool
    accurary = tf.reduce_mean(tf.cast(correct, np.float32))
    return accurary



In [4]:
def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i == 0) | ((i + 1) % verbose == 0):
            acc = predict(X, Y).numpy()
            loss = cost_fn(X, Y).numpy()
            print("Steps: {} Loss: {}, Acc: {}%".format(i + 1, loss, int(acc * 100)))
    
fit(x_data, Y_one_hot)

Steps: 1 Loss: 5.075472831726074, Acc: 8%
Steps: 100 Loss: 0.6965628266334534, Acc: 79%
Steps: 200 Loss: 0.43151724338531494, Acc: 87%
Steps: 300 Loss: 0.31091007590293884, Acc: 91%
Steps: 400 Loss: 0.2398422360420227, Acc: 93%
Steps: 500 Loss: 0.19407987594604492, Acc: 95%
Steps: 600 Loss: 0.16291461884975433, Acc: 96%
Steps: 700 Loss: 0.14055044949054718, Acc: 99%
Steps: 800 Loss: 0.12374600023031235, Acc: 100%
Steps: 900 Loss: 0.11063815653324127, Acc: 100%
Steps: 1000 Loss: 0.10010698437690735, Acc: 100%
